In [1]:
import cv2
import mediapipe as mp
import joblib
import pyttsx3

2025-04-03 04:13:30.454440: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-03 04:13:30.482935: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-03 04:13:30.482975: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-03 04:13:30.484022: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-03 04:13:30.489617: I tensorflow/core/platform/cpu_feature_guar

In [2]:
model_path = "/home/abhinav/Desktop/AIProjects/General Projects/SLT/sign_recognition/models/rndm_forest.pkl"

In [3]:
model = joblib.load(model_path)

In [4]:
min_det_conf = 0.5
min_track_conf = 0.5
precision = 3
frame_speed_control_var = 100

labels_dict = {}

In [5]:
cnt = 65
for i in range(0, 42):
    if i == 37 or i == 38:
        labels_dict[i] = ""
    elif i == 39:
        labels_dict[i] = "Special"
    elif i == 40:
        labels_dict[i] = "Special_end"
    elif i == 41:
        labels_dict[i] = "End"
    elif i < 10:
        labels_dict[i] = str(i)
    elif i == 14:
        continue
    else:
        labels_dict[i] = chr(cnt)
        cnt += 1


In [6]:
# for i in labels_dict:
#     print("Key and value: ", i, " ", labels_dict[i])

In [7]:
def get_short(x, n):
    x = str(x)
    al = x.split(".")
    c = al[0] + "." + al[1][:n]
    return float(c)

In [8]:
engine = pyttsx3.init()

In [9]:
cap = cv2.VideoCapture(0)

mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=min_det_conf, min_tracking_confidence=min_track_conf)

sp = False
name = ""

while True:
    
    ret, frame = cap.read()
    


    # img = cv2.imread(frame)
    img = frame
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    

    results = hands.process(img)

    right_detected = False
    left_detected = False

    left_data = []
    right_data = []

    data_point = []

    if results.multi_hand_landmarks is not None:
        for idx, (hand, hand_label) in enumerate(zip(results.multi_hand_landmarks, results.multi_handedness)):
            


            if hand_label.classification is not None:
                hand_class = "Right" if hand_label.classification[0].label == "Left" else "Left"

                if hand_class == "Right":

                    right_detected = True

                    for i in range(21):
                        x1 = get_short(hand.landmark[i].x, precision)
                        y1 = get_short(hand.landmark[i].y, precision)
                        z1 = get_short(hand.landmark[i].z, precision)

                        right_data.extend([float(x1), float(y1), float(z1)])
                    

                else:

                    left_detected = True
                
                    for i in range(21):
                        x1 = get_short(hand.landmark[i].x, precision)
                        y1 = get_short(hand.landmark[i].y, precision)
                        z1 = get_short(hand.landmark[i].z, precision)

                        left_data.extend([float(x1), float(y1), float(z1)])
    # del results
    # del hands
    # del mp_hands
    # del img

    if left_data == []:
        left_data = [0 for i in range(63)]
    if right_data == []:
        right_data = [0 for i in range(63)]


    # print("Right data: ", right_data)
    # print("Left data: ", left_data)
    # print("Label: ", data_label)

    if left_data == [] and right_data == []:
        print("No hands detected")

    elif left_data != [] or right_data != []:

        data_point = left_data + right_data
        # print(data_point)
        prediction = ""
        if len(data_point) == 126:
            prediction = labels_dict[model.predict([data_point])[0]]
            # print("Prediction: ", prediction)
        
        if prediction == "Special":
            sp = True
            # cv2.putText(frame, prediction, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.3, (150, 100, 50), 3,
            #     cv2.LINE_AA)
        elif prediction == "End":
            if sp == True:
                engine.say(name)
                engine.runAndWait()
            cv2.putText(frame, name, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.3,  (150, 100, 50), 3,
                cv2.LINE_AA)
            sp = False
        if sp == True:
            if prediction != "Special" and prediction!= "Special_end":
                # if name != "" and prediction not in ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]:
                if name != "":
                    # if prediction != name[-1]:
                    if prediction not in name:
                        name += prediction
                else:
                    # if prediction not in ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]:
                    name += prediction
        elif sp == False and prediction == "End":
            cv2.putText(frame, name, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.3,  (150, 100, 50), 3,
                cv2.LINE_AA)
        else:    
            name = ""
            if prediction != "End":
                # engine.say(prediction)
                # engine.runAndWait()
                cv2.putText(frame, prediction, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.3,  (150, 100, 50), 3,
                        cv2.LINE_AA)

        # cv2.putText(frame, prediction, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.3, (150, 90, 5), 3,
        #             cv2.LINE_AA)

    cv2.imshow("Frame", frame)
    
    if cv2.waitKey(frame_speed_control_var) == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()

I0000 00:00:1743633812.835880    8876 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1743633812.866592    8971 gl_context.cc:344] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 535.183.01), renderer: NVIDIA GeForce RTX 3050 Laptop GPU/PCIe/SSE2
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
/home/abhinav/.local/lib/python3.10/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
/home/abhinav/.local/lib/python3.10/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
/home/abhinav/.local/lib